In [1]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.metrics import f1_score, r2_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB

In [2]:
data = pd.read_excel('NFI_data/modified files/big_merge_meteo_sat.xlsx')
data.drop('Unnamed: 0',axis=1,inplace=True)

### Selecting features and modeling

Other models were attempted at the same time by other members of the team. Among them :<br>
- Logistic Regression (TAUX_COUV_RAJ)
- Linear Regression, Lasso, Ridge (ACCR & SURF_TER_HA)

Hist Gradient Boosting was chosen in part for its ability to easily handle missing values.

Creating 2 new features to simulate satellite data we don't have

In [3]:
# adding aridity index
data["AI"] = data['PRCP_GROWTH'] / data['TAVE_GROWTH']
# adding H/D index
data["H_D"] = data['HAUTEUR_ARBRE'] / data['DBH']

#### 1. Regression targeting <a href="https://en.wikipedia.org/wiki/Basal_area">basal area</a> or forest regeneration coverage rate.

In [4]:
features_list = ['ALT', 'SLOPE25','AGE_PPL','PRCP_G_S','PRCP_S_S','TAVE_GROWTH','TAVE_AVG','STR_PPL','ORIENTATION','TAILLE_PPL','LFI', 'INTENSITE_EXPLOIT', 'MODE_REGEN', 'TYP_RAJ_PPL',
                    'RELIEF','ESPECE_DOM','FEU_RES','TYPE_FORET','TRACES_FEU','TYPE_FORET305','PRODREG','NDVI', 'EVI', 'NDMI', 'NDWI','DSWI','AI','H_D','HT_VEG', 'REBOISEMENT_AN',
                    'PERI_CHENAUX', 'PERI_COULEES', 'PERI_AVALANCH', 'PERI_CHUTES']
target_variable =  ['TAUX_COUV_RAJ']

dcopy = data.loc[data['TAUX_COUV_RAJ'].notnull()]
X = dcopy.loc[:, features_list]
Y = dcopy.loc[:, target_variable]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

In [6]:
#preprocessing cell
numeric_features = ['ALT', 'SLOPE25','AGE_PPL','PRCP_G_S','PRCP_S_S','TAVE_GROWTH','TAVE_AVG','NDVI', 'EVI', 'NDMI', 'NDWI','DSWI','AI','H_D']
categorical_features = ['STR_PPL','ORIENTATION','LFI','RELIEF','ESPECE_DOM','FEU_RES','TYPE_FORET','TYPE_FORET305','TRACES_FEU','PRODREG','INTENSITE_EXPLOIT','MODE_REGEN','TYP_RAJ_PPL',
                        'PERI_CHENAUX', 'PERI_COULEES', 'PERI_AVALANCH', 'PERI_CHUTES']
ordinal_categorical_features = ['HT_VEG', 'REBOISEMENT_AN', 'TAILLE_PPL']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')
ordinal_transformer = OrdinalEncoder(encoded_missing_value=0)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ordi', ordinal_transformer, ordinal_categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [7]:
hgbr = HistGradientBoostingRegressor(max_iter= 200,min_samples_leaf= 4,max_leaf_nodes= 50, max_depth= 16,random_state=123)
hgbr.fit(X_train, Y_train.values.ravel())

HistGradientBoostingRegressor(max_depth=16, max_iter=200, max_leaf_nodes=50,
                              min_samples_leaf=4, random_state=123)

Predictions on SURF_TER_HA (basal area)

In [52]:
train_pred = hgbr.predict(X_train)
test_pred = hgbr.predict(X_test)

print("r2-score on train set : ", r2_score(Y_train, train_pred))
print("r2-score on test set : ", r2_score(Y_test, test_pred))

r2-score on train set :  0.9014736438268208
r2-score on test set :  0.47013498416860455


Predictions on TAUX_COUV_RAJ (forest regeneration coverage rate)

In [8]:
train_pred = hgbr.predict(X_train)
test_pred = hgbr.predict(X_test)

print("r2-score on train set : ", r2_score(Y_train, train_pred))
print("r2-score on test set : ", r2_score(Y_test, test_pred))

r2-score on train set :  0.9821517460833592
r2-score on test set :  0.8705672119457397


Optimizing hyperparameters with gridsearch

In [32]:
from sklearn.model_selection import GridSearchCV

params = {
    'learning_rate': [0.1,0.05,0.2],
    'max_leaf_nodes': [20,30,40,50,60],
    'max_depth' : [6,8,10,12,16],
    'min_samples_leaf' : [2,4,6,8,10],
    'max_iter' : [75,100,125,150]
}

gridsearch = GridSearchCV(hgbr, param_grid = params, n_jobs=-1, cv = 7,scoring='r2') 
gridsearch.fit(X_train, Y_train.values.ravel())
print("Best: %f using %s" % (gridsearch.best_score_, gridsearch.best_params_))

Best: 0.416188 using {'learning_rate': 0.1, 'max_depth': 16, 'max_iter': 150, 'max_leaf_nodes': 50, 'min_samples_leaf': 2}


In [33]:
train_pred = gridsearch.predict(X_train)
test_pred = gridsearch.predict(X_test)

print("r2-score on train set : ", r2_score(Y_train, train_pred))
print("r2-score on test set : ", r2_score(Y_test, test_pred))

r2-score on train set :  0.8346012655762125
r2-score on test set :  0.4592261677384031


#### 2. Multi-class classification targeting forest regeneration coverage rate.

In [5]:
target_variable =  ['TAUX_COUV_RAJ']

dcopy = data.loc[data['TAUX_COUV_RAJ'].notnull()]
X = dcopy.loc[:, features_list]
#X = dcopy.loc[:, features_list].notnull()
Y = dcopy.loc[:, target_variable]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

In [9]:
#re-run preprocessing cell above

In [347]:
cnb = CategoricalNB()
cnb.fit(X_train, Y_train.values.ravel())

train_pred = cnb.predict(X_train)
test_pred = cnb.predict(X_test)

print("f1-score on train set : ", f1_score(Y_train, train_pred,average='weighted'))
print("f1-score on test set : ", f1_score(Y_test, test_pred,average='weighted'))

f1-score on train set :  0.4386664287953212
f1-score on test set :  0.4473877293749791


In [28]:
hgbc = HistGradientBoostingClassifier(learning_rate=0.2,max_depth=20,max_iter=50, max_leaf_nodes=16,min_samples_leaf=4,random_state=123)
hgbc.fit(X_train, Y_train.values.ravel())

HistGradientBoostingClassifier(learning_rate=0.2, max_depth=20, max_iter=50,
                               max_leaf_nodes=16, min_samples_leaf=4,
                               random_state=123)

In [29]:
train_pred = hgbc.predict(X_train)
test_pred = hgbc.predict(X_test)

print("f1-score on train set : ", f1_score(Y_train, train_pred,average='weighted'))
print("f1-score on test set : ", f1_score(Y_test, test_pred,average='weighted'))

f1-score on train set :  0.9244015131217471
f1-score on test set :  0.6146957225073904


In [9]:
print(classification_report(Y_test,test_pred))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       495
           1       1.00      1.00      1.00        78
           2       0.62      0.70      0.66       457
           3       0.42      0.40      0.41       444
           4       0.33      0.35      0.34       261
           5       0.22      0.15      0.18       124
           6       0.20      0.17      0.18        64

    accuracy                           0.62      1923
   macro avg       0.54      0.54      0.54      1923
weighted avg       0.61      0.62      0.61      1923



TAUX_COUV_RAJ values : 
- -1 : N/A
- 0 : No regeneration DG < 1%
- 2 : Regeneration coverage rate 1 to 9
- 3 : Regeneration coverage rate 10 to 25
- 4 : Regeneration coverage rate 26 to 50% 
- 5 : Regeneration coverage rate 51 to 75
- 6 : Regeneration coverage rate 76 to 100%.

The presence of a significant number of N/A probably doesn't help us much (though simply taking them out worsens the predictions).

These attempts were made with features that don't require any recent data collection in the field. All efforts of the team on various other models with a similar approach did not yield good results.

##### 3. Adding field data to the mix

In [84]:
features_list = ['ALT', 'SLOPE25','AGE_PPL','PRCP_G_S','PRCP_S_S','TAVE_GROWTH','TAVE_AVG','STR_PPL','ORIENTATION','TAILLE_PPL','LFI', 'INTENSITE_EXPLOIT', 'MODE_REGEN', 'TYP_RAJ_PPL',
                    'RELIEF','ESPECE_DOM','FEU_RES','TYPE_FORET','TRACES_FEU','TYPE_FORET305','PRODREG','NDVI', 'EVI', 'NDMI', 'NDWI','DSWI','AI','H_D','HT_VEG', 'REBOISEMENT_AN',
                    'PERI_CHENAUX', 'PERI_COULEES', 'PERI_AVALANCH', 'PERI_CHUTES', 'V_ALL']
target_variable =  ['SURF_TER_HA']

dcopy = data.loc[data['SURF_TER_HA'].notnull()]
X = dcopy.loc[:, features_list]
Y = dcopy.loc[:, target_variable]

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

In [86]:
numeric_features = ['ALT', 'SLOPE25','AGE_PPL','PRCP_G_S','PRCP_S_S','TAVE_GROWTH','TAVE_AVG','NDVI', 'EVI', 'NDMI', 'NDWI','DSWI','AI','H_D', 'V_ALL']
categorical_features = ['STR_PPL','ORIENTATION','LFI','RELIEF','ESPECE_DOM','FEU_RES','TYPE_FORET','TYPE_FORET305','TRACES_FEU','PRODREG','INTENSITE_EXPLOIT','MODE_REGEN','TYP_RAJ_PPL',
                        'PERI_CHENAUX', 'PERI_COULEES', 'PERI_AVALANCH', 'PERI_CHUTES']
ordinal_categorical_features = ['HT_VEG', 'REBOISEMENT_AN', 'TAILLE_PPL']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')
ordinal_transformer = OrdinalEncoder(encoded_missing_value=0)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ordi', ordinal_transformer, ordinal_categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [87]:
hgbr = HistGradientBoostingRegressor(max_iter= 150,min_samples_leaf= 4,max_leaf_nodes= 40, max_depth= 16)
hgbr.fit(X_train, Y_train.values.ravel())

HistGradientBoostingRegressor(max_depth=16, max_iter=150, max_leaf_nodes=40,
                              min_samples_leaf=4)

In [88]:
train_pred = hgbr.predict(X_train)
test_pred = hgbr.predict(X_test)

print("r2-score on train set : ", r2_score(Y_train, train_pred))
print("r2-score on test set : ", r2_score(Y_test, test_pred))

r2-score on train set :  0.9922817588459242
r2-score on test set :  0.9603295062821063


Adding features that include measurements from diameter of trees and total volume yields much better results, which is not surprising as the two are mathematically connected to the <a href="https://en.wikipedia.org/wiki/Basal_area">basal area</a> we are looking for.<br>
Meteorological and satellite data (caveat : at the resolution available for public research) are not a sufficient substitute for field data collection.
<br>

Given the very few different observations for each feature, we can't get good results from time series predictions.<br>
Our few tests proved even less effective than simply keeping the values from the previous collection campaign !

In [57]:
LFI1 = data[data['LFI']=='LFI1']
LFI2 = data[data['LFI']=='LFI2']
LFI3 = data[data['LFI']=='LFI3']
LFI4 = data[data['LFI']=='LFI4']

pred = LFI3['SURF_TER_HA']
real = LFI4['SURF_TER_HA']

print("r2-score on train set : ", r2_score(real, pred))

r2-score on train set :  0.6275905847214268


In [56]:
pred = LFI3[LFI3['TAUX_COUV_RAJ']>=0]['TAUX_COUV_RAJ']
real = LFI4[LFI4['TAUX_COUV_RAJ']>=0]['TAUX_COUV_RAJ']

print("r2-score on train set : ", f1_score(real, pred,average="weighted"))

r2-score on train set :  0.4587315517255523


Rate of increase in basal area

In [80]:
avg_increase = LFI1['SURF_TER_HA'].mean(), LFI2['SURF_TER_HA'].mean(), LFI3['SURF_TER_HA'].mean(), LFI4['SURF_TER_HA'].mean()
avg_increase_rate = np.mean([(avg_increase[1] - avg_increase[0])/avg_increase[0],(avg_increase[2] - avg_increase[1])/avg_increase[1], (avg_increase[3] - avg_increase[2])/avg_increase[2]])
print(f"Basal area has a {round(avg_increase_rate*100,2)} % average increase rate across all campaigns")

Basal area has a 5.25 % average increase rate across all campaigns
